<p style="border:2px solid black"> </p>
<span style="font-family:Lucida Bright;">
<p style="margin-bottom:1cm"></p>
<center>
<font size="7"><b>Social Data Analysis and Visualization</b></font>
<p style="margin-bottom:1cm"></p>
<font size="3"><b>Final Project</b></font>
<p style="margin-bottom:1cm"></p>
<font size="6"><b>Demographics of Copenhagen</b></font>
<p style="margin-bottom:0.8cm"></p>
<font size="3"><b>Wojciech Mazurkiewicz, DTU, 14 May 2021</b></font>
<p style="margin-bottom:1.5cm"></p>
<font size="6"><b>Data Analysis and Visualization</b></font>
<br>
<font size="3"><b></b></font>
</center>
<p style="margin-bottom:0.7cm"></p>
<p style="border:2px solid black"> </p>

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#How-to-read-this-notebook" data-toc-modified-id="How-to-read-this-notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>How to read this notebook</a></span></li><li><span><a href="#Introduction" data-toc-modified-id="Introduction-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Initialization" data-toc-modified-id="Initialization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Initialization</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Country-of-origin-(without-district-information)" data-toc-modified-id="Country-of-origin-(without-district-information)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Country of origin (without district information)</a></span><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Show-data" data-toc-modified-id="Show-data-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Show data</a></span><ul class="toc-item"><li><span><a href="#Plot-the-population-of-top-10-countries-of-origin-for-each-year" data-toc-modified-id="Plot-the-population-of-top-10-countries-of-origin-for-each-year-5.2.1"><span class="toc-item-num">5.2.1&nbsp;&nbsp;</span>Plot the population of top 10 countries of origin for each year</a></span><ul class="toc-item"><li><span><a href="#By-number-of-people" data-toc-modified-id="By-number-of-people-5.2.1.1"><span class="toc-item-num">5.2.1.1&nbsp;&nbsp;</span>By number of people</a></span></li><li><span><a href="#By-percentage-of-the-total-population-of-Copenhagen" data-toc-modified-id="By-percentage-of-the-total-population-of-Copenhagen-5.2.1.2"><span class="toc-item-num">5.2.1.2&nbsp;&nbsp;</span>By percentage of the total population of Copenhagen</a></span></li><li><span><a href="#Interactive-bar-graph-of-2020-with-plotly" data-toc-modified-id="Interactive-bar-graph-of-2020-with-plotly-5.2.1.3"><span class="toc-item-num">5.2.1.3&nbsp;&nbsp;</span>Interactive bar graph of 2020 with plotly</a></span></li></ul></li><li><span><a href="#Plot-the-relationship-between-Danes-and-non-Danes" data-toc-modified-id="Plot-the-relationship-between-Danes-and-non-Danes-5.2.2"><span class="toc-item-num">5.2.2&nbsp;&nbsp;</span>Plot the relationship between Danes and non-Danes</a></span><ul class="toc-item"><li><span><a href="#By-number-of-people" data-toc-modified-id="By-number-of-people-5.2.2.1"><span class="toc-item-num">5.2.2.1&nbsp;&nbsp;</span>By number of people</a></span></li><li><span><a href="#By-percentage-of-the-total-population-of-Copenhagen" data-toc-modified-id="By-percentage-of-the-total-population-of-Copenhagen-5.2.2.2"><span class="toc-item-num">5.2.2.2&nbsp;&nbsp;</span>By percentage of the total population of Copenhagen</a></span></li><li><span><a href="#Interactive-graph-with-plotly" data-toc-modified-id="Interactive-graph-with-plotly-5.2.2.3"><span class="toc-item-num">5.2.2.3&nbsp;&nbsp;</span>Interactive graph with plotly</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Citizenship" data-toc-modified-id="Citizenship-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Citizenship</a></span><ul class="toc-item"><li><span><a href="#Load-data" data-toc-modified-id="Load-data-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Load data</a></span></li><li><span><a href="#Get-percentage-of-district-population-by-citizenship" data-toc-modified-id="Get-percentage-of-district-population-by-citizenship-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Get percentage of district population by citizenship</a></span></li><li><span><a href="#Show-data-for-2020" data-toc-modified-id="Show-data-for-2020-6.3"><span class="toc-item-num">6.3&nbsp;&nbsp;</span>Show data for 2020</a></span></li><li><span><a href="#Create-a-plotly-barplot-showing-the-percentages-of-population-by-citizenship-for-all-districts-in-2020" data-toc-modified-id="Create-a-plotly-barplot-showing-the-percentages-of-population-by-citizenship-for-all-districts-in-2020-6.4"><span class="toc-item-num">6.4&nbsp;&nbsp;</span>Create a plotly barplot showing the percentages of population by citizenship for all districts in 2020</a></span></li><li><span><a href="#Create-a-geoplot-of-the-percentage-of-non-Danes-by-district-over-time" data-toc-modified-id="Create-a-geoplot-of-the-percentage-of-non-Danes-by-district-over-time-6.5"><span class="toc-item-num">6.5&nbsp;&nbsp;</span>Create a geoplot of the percentage of non-Danes by district over time</a></span><ul class="toc-item"><li><span><a href="#All-non-Danes" data-toc-modified-id="All-non-Danes-6.5.1"><span class="toc-item-num">6.5.1&nbsp;&nbsp;</span>All non-Danes</a></span></li><li><span><a href="#Western-non-Danes" data-toc-modified-id="Western-non-Danes-6.5.2"><span class="toc-item-num">6.5.2&nbsp;&nbsp;</span>Western non-Danes</a></span></li><li><span><a href="#Non-western-non-Danes" data-toc-modified-id="Non-western-non-Danes-6.5.3"><span class="toc-item-num">6.5.3&nbsp;&nbsp;</span>Non-western non-Danes</a></span></li></ul></li></ul></li><li><span><a href="#Marital-status" data-toc-modified-id="Marital-status-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Marital status</a></span></li><li><span><a href="#Family-type-and-children" data-toc-modified-id="Family-type-and-children-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Family type and children</a></span></li><li><span><a href="#Income" data-toc-modified-id="Income-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Income</a></span><ul class="toc-item"><li><span><a href="#Load-demographic-data" data-toc-modified-id="Load-demographic-data-9.1"><span class="toc-item-num">9.1&nbsp;&nbsp;</span>Load demographic data</a></span></li><li><span><a href="#Load-geodata" data-toc-modified-id="Load-geodata-9.2"><span class="toc-item-num">9.2&nbsp;&nbsp;</span>Load geodata</a></span></li><li><span><a href="#Get-data-about-the-average-income" data-toc-modified-id="Get-data-about-the-average-income-9.3"><span class="toc-item-num">9.3&nbsp;&nbsp;</span>Get data about the average income</a></span></li><li><span><a href="#Show-geoplot" data-toc-modified-id="Show-geoplot-9.4"><span class="toc-item-num">9.4&nbsp;&nbsp;</span>Show geoplot</a></span></li></ul></li><li><span><a href="#Life-span" data-toc-modified-id="Life-span-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Life span</a></span></li><li><span><a href="#Population-movement-data" data-toc-modified-id="Population-movement-data-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>Population movement data</a></span></li><li><span><a href="#Dwellings" data-toc-modified-id="Dwellings-12"><span class="toc-item-num">12&nbsp;&nbsp;</span>Dwellings</a></span></li></ul></div>

# How to read this notebook

<p style="border:2px solid black"> </p>

Please note that the pre-rendered outputs will first display properly when the notebook is __trusted__.

If you are viewing the HTML-version of the notebook and would like to download the .ipynm file, you can do it [here](https://social-data-analysis-and-visualization-final-project.s3.eu-central-1.amazonaws.com/data_visualization.ipynb).

# Introduction

<p style="border:2px solid black"> </p>

This notebook aims to describe the process of analyzing and visualizing of the data about different demographical quantities for Copenhagen, including: 

1. year vs population by country of origin
1. year and district vs population by citizenship (Danes vs. western and non-western non-Danes)
1. year and district vs population by marital status
1. year and district vs number of families by family type and number of children
1. year and district vs average income
1. year and district vs average life span
1. year and district vs population movements (immigration, births, deaths, etc.)  

Each demographical quantity is described in its own section. However, as mentioned in the Motivation section of the [Explainer Notebook](https://social-data-analysis-and-visualization-final-project.s3.eu-central-1.amazonaws.com/explainer_notebook.html), the time constraints made it difficult for me to create analyses of all the datasets and quantities I have gathered for this project, and I have therefore taken the approach of letting my webpage story dictate the choice of the analysis to perform, and let results of this analysis inspire the continuation of the story. Therefore, in this section, you will find (almost) solely the analyses of the quantities mentioned on the [webpage](https://social-data-analysis-and-visualization-final-project.s3.eu-central-1.amazonaws.com/website_main.html).

I have, however, decided to let the structure of this notebook reflect the quantities of clean data that we have at our disposal, and have therefore kept the sections dedicated to each quantity, even though many of them are empty. I hope this will make it easier to keep an overview of the work that still needs to be done, and make it easier to continue the work on the project.

# Initialization

<p style="border:2px solid black"> </p>

The code in this notebook can only run correctly after running the initialization procedure defined in the notebook [Initialization](https://social-data-analysis-and-visualization-final-project.s3.eu-central-1.amazonaws.com/initialization.html), which contains the imports, configuration, and function definitions. Let's run it now:

In [ ]:
%run ./initialization.ipynb

# Load data

<p style="border:2px solid black"> </p>

Let's load the object containing all dataframes containing clean data we have created in the process of [loading and cleaning](https://social-data-analysis-and-visualization-final-project.s3.eu-central-1.amazonaws.com/data_loading_and_cleaning.html). 

In [ ]:
cph_data = CphData()

# Country of origin (without district information)

<p style="border:2px solid black"> </p>

## Load data

Let's load the data about the population of Copenhagen by the country of origin:

In [ ]:
# Load the data containing information about the population
# of Copenhagen by the country of origin.
df_country = cph_data.country_of_origin

# Show dataframe.
display(df_country)

## Show data

Let's visualize the data. We will start by removing data for 2008 to set the number of available years to 12 for convenience of viewing.

In [ ]:
df_country = df_country[df_country['Year'] != 2008]
display(df_country)

Now, let's create a dataframe where each row will represent a year and each column population of a given country of origin:

In [ ]:
# Create a dataframe where each row represents a year and each columnn
df_country_vs_year = (
    df_country
    .loc[:,
         ['Year', 'Country of origin', 'Number of people']]
    .groupby(['Year', 'Country of origin'])
    .sum()
    .unstack(level=1)
    .droplevel(0, axis=1)
)

# Save the dataframe to pickle.
df_country_vs_year.to_pickle(path_data_clean_root / 'df_country_of_origin_vs_year.pkl')

# Show the results.
display(df_country_vs_year)

### Plot the population of top 10 countries of origin for each year

We start by getting the list of years, from most recent to least recent:

In [ ]:
# Get a list of countries represented in the dataframe,
# Sorted by summarized number of people over the years
countries_sorted_by_number_of_people = (
    df_country_vs_year
    .sum()
    .sort_values(ascending=False)
    .index
    .to_list()
)

# Show the countries sorted by population
# display(countries_sorted_by_number_of_people)

# Years from biggest to smallest.
years = df_country_vs_year.index.to_list()
years.sort(reverse=True)

# Show the results.
display(years)

#### By number of people

Now we can show the total population of each of the countries for all the years:

In [ ]:
# Get the number of years.
n_years = len(years)

# Get the number of counries.
n_countries = len(countries_sorted_by_number_of_people)

# Get the number of plots.
n_plots = len(years)

# Define the plot grid.
n_plot_columns = 3
n_plot_rows = int(np.ceil(n_plots / n_plot_columns))

# Create a figure for the plots.
figure, all_axes = plt.subplots(
    n_plot_rows, n_plot_columns,
    figsize=(5 * n_plot_columns + 2, 5 * n_plot_rows),
    gridspec_kw={'hspace': 0.3}
)

# Get the handles of the bottom axes'.
bottom_axes = all_axes[-1, :]

# Define colors.
n_countries_to_map = 15
color_palette = sns.color_palette("hls",
                                  n_colors=n_countries_to_map)

# Map the colors to countries.
color_mapping = {country: color
                 for country, color
                 in zip(countries_sorted_by_number_of_people[:n_countries_to_map],
                        color_palette)}

# Plot.
for idx, (year, axes) in enumerate(zip(years, all_axes.ravel()[:n_plots])):

    total = df_country_vs_year.at[year, 'Total']

    sns.barplot(
        data=(
            df_country_vs_year
            .loc[year, ~df_country_vs_year.columns.isin(['Total'])]
            .sort_values(ascending=False)
            .head(10)
            .div(1e3)
            .reset_index()
        ),
        x='Country of origin',
        y=year,
        ax=axes,
        palette=color_mapping)

    # Set the title of the plot.
    axes.set_title(year, y=0.88)
    axes.set_xlabel('')
    axes.set_ylabel('')
    axes.set_ylim([axes.get_ylim()[0], total / 1e3 * 1.2])
    
    draw_threshold(total * 1e-3, axes, title=f'Total: {total:,.0f}')

    # Rotate x tick labels.
    plt.setp(
        axes.get_xticklabels(),
        rotation=45,
        ha='right',
        va='top',
    )

    # Apply the standard formatting.
    format_axes_annotation(axes)

# Annotate the figure.
# figure_x_label(figure, 'Day of week', y_position=0.06)
figure_y_label(figure, 'Population in Copenhagen region [thousands]', x_position=0.08)
figure.suptitle('10 most represented countries of origin in Copenhagen', 
                size=24,
                y=0.91)
figure.set_facecolor("white")

We can see that Danes vastly outnumber any other nationality group in Copenhagen.

#### By percentage of the total population of Copenhagen

Instead of showing the absolute numbers, it might be more interesting to see the populations as a fraction of the total population of Copenhagen:

In [ ]:
# Years from biggest to smallest.
years = df_country_vs_year.index.to_list()
years.sort(reverse=True)

# Get the number of counries.
n_countries = len(countries_sorted_by_number_of_people)

# Get the number of plots.
n_plots = len(years)

# Define the plot grid.
n_plot_columns = 3
n_plot_rows = int(np.ceil(n_plots / n_plot_columns))

# Create a figure for the plots.
figure, all_axes = plt.subplots(
    n_plot_rows, n_plot_columns,
    sharey='all',
    figsize=(5 * n_plot_columns + 2, 5 * n_plot_rows),
    gridspec_kw={'hspace': 0.3}
)

# Get the handles of the bottom axes'.
bottom_axes = all_axes[-1, :]

# Define colors.
n_countries_to_map = 15
color_palette = sns.color_palette("hls",
                                  n_colors=n_countries_to_map)

# Map the colors to countries.
color_mapping = {country: color
                 for country, color
                 in zip(countries_sorted_by_number_of_people[:n_countries_to_map],
                        color_palette)}

# Plot.
for idx, (year, axes) in enumerate(zip(years, all_axes.ravel()[:n_plots])):
    
    # The total number of people in Copenhagen.
    total = df_country_vs_year.at[year, 'Total']
    
    # Show the barplot.
    sns.barplot(
        data=(
            df_country_vs_year
            .loc[year, ~df_country_vs_year.columns.isin(['Total'])]
            .sort_values(ascending=False)
            .head(10)
            .mul(100 / total)
            .reset_index()
        ),
        x='Country of origin',
        y=year,
        ax=axes,
        palette=color_mapping)

    # Set the title of the plot.
    axes.set_title(year, y=0.9)
    axes.set_xlabel('')
    axes.set_ylabel('')

    # Rotate x tick labels.
    plt.setp(
        axes.get_xticklabels(),
        rotation=45,
        ha='right',
        va='top',
    )

    # Apply the standard formatting.
    format_axes_annotation(axes)

# Annotate the figure.
# figure_x_label(figure, 'Day of week', y_position=0.06)
figure_y_label(figure, r'% of polulation in Copenhagen region', x_position=0.08)
figure.suptitle('10 most represented countries of origin in Copenhagen', 
                size=24,
                y=0.91)

figure.set_facecolor("white")

figure.savefig(path_results_root / 'country_of_origin_10_most_represented.png', bbox_inches='tight')

From this graph, it is possible to see the fraction of the population of Copenhagen constituted by the Danes, but it is virtually impossible to estimate the fraction of any other nationality. To do that, let's create an interactive plot using plotly.

#### Interactive bar graph of 2020 with plotly

We can also make the graph interactive by creating it using plotly:

In [ ]:
# Define the year of interest.
year = 2020

# Create a dataframe that shows % of population by country of origin
# in the chosen year.
df_country_2020 = (
    df_country_vs_year
    .loc[year, ~df_country_vs_year.columns.isin(['Total'])]
    .sort_values(ascending=False)
    .head(10)
    .mul(100 / df_country_vs_year.at[year, 'Total'])
    .reset_index()
    .rename(columns={year: '% of population'})
)

# Create the plotly bar graph.
fig = px.bar(
    df_country_2020,
    x='Country of origin',
    y='% of population',
#     barmode='group',
    color='Country of origin',
    color_discrete_sequence=px.colors.qualitative.D3
)

# Update the layout of the figure.
fig.update_layout(
    dict(plot_bgcolor='rgb(256, 256, 256)',
         paper_bgcolor='rgb(256, 256, 256)',
         title=dict(text="Population of Copenhagen in 2020",
                    x=0.5,
                    xanchor='center',
                    yanchor='top'),
         xaxis=dict(tickmode='linear',
                    #                         tick0=0.5
                    dtick=1),
         showlegend=False
#          annotations=caption_attributes(caption, y),
#          margin=margin
         )
)

# Show the figure
fig.show()

We can now read directly off the graph that 74.29% of the population of Copenhagen in 2020 was of Danish origin. The second largest national group, the Pakistanis, constituted only 1.39% of the population that year.

### Plot the relationship between Danes and non-Danes 

Let's create a dataframe where we will divide all countries of origin into two categories: Danes and non-Danes:

In [ ]:
# Get the number of counries.
n_countries = len(countries_sorted_by_number_of_people)

# Get the number of plots.
n_plots = len(years)

# Define the plot grid.
n_plot_columns = 3
n_plot_rows = int(np.ceil(n_plots / n_plot_columns))

# Create a dataframe with data for danes vs non-danes.
df_danes_vs_non_danes = (
    df_country_vs_year
    .loc[:, 'Denmark']
    .to_frame('Danes')
)

df_danes_vs_non_danes['Non-danes'] = (
    df_country_vs_year
    .loc[:, ~df_country_vs_year.columns.isin(['Total', 'Denmark'])]
    .sum(axis=1)
    .to_frame('Non-danes')
)

df_danes_vs_non_danes[['% Danes', '% Non-danes']] = (
    df_danes_vs_non_danes[['Danes', 'Non-danes']]
    .div(df_danes_vs_non_danes.sum(axis=1), axis=0)
    .mul(100)
)


# Save the dataframe to a file
df_danes_vs_non_danes.to_pickle(
    path_data_clean_root / 'df_danes_vs_non_danes.pkl')


# Show the dataframe
display(df_danes_vs_non_danes)

####  By number of people

Let's show how the number of Danes and non-Danes have developed over the years.

In [ ]:
# Create a figure for the plots.
figure, axes = plt.subplots(figsize=(15, 8))

# Create the barplot for each year.
sns.barplot(data=(df_danes_vs_non_danes[['Danes', 'Non-danes']]
                  .div(1e3)
                  .reset_index()
                  .melt(id_vars='Year',
                        var_name='Origin',
                        value_name='Number of people')),
            x='Year',
            y='Number of people',
            hue='Origin',
            ax=axes)

# Total population over the years.
total = (
    df_danes_vs_non_danes[['Danes', 'Non-danes']]
    .div(1e3)
    .sum(axis=1)
    .to_numpy()
)

# Plot total population.
axes.plot(df_danes_vs_non_danes[['Danes', 'Non-danes']].div(1e3).sum(axis=1).to_numpy(),
          color='red')

# Annotate.
axes.set_ylabel('Number of people [thousands]')
axes.text(0, total[0] + 20, 'Total population', rotation = 5, size=14)

# Format plot.
format_axes(axes)
format_axes_annotation(axes)
figure.set_facecolor("white")

# Save the plot.
figure.savefig(path_results_root / 'total_population_danes_vs_non_danes_absolute.png')


Here, we can see the population of Danes and non-Danes in absolute numbers, but it can be difficult to estimate the proportions between the two groups. To make it easier, let's express the size of each population in terms of the fraction of the total population of Copenhagen.

#### By percentage of the total population of Copenhagen

As mentioned above, it might be easier to get an intuition about the numbers if the sizes of the populations are expressed as fractions of the total population of Copenhagen:

In [ ]:
# Create a figure for the plots.
figure, axes = plt.subplots(figsize=(15, 8))

# Create the barplot for each year.
sns.barplot(data=(df_danes_vs_non_danes[['% Danes', '% Non-danes']]
                  .rename(columns={'% Danes': 'Danes',
                                   '% Non-danes': 'Non-danes'})
                  .reset_index()
                  .melt(id_vars='Year',
                        var_name='Origin',
                        value_name='% of population')),
            x='Year',
            y='% of population',
            hue='Origin',
            ax=axes)

# Set axes limits (to make room for the legend)
axes.set_ylim((0, 90))
axes.set_title('Population of Copenhagen by country of origin')

format_axes(axes)
format_axes_annotation(axes)
figure.set_facecolor("white")

# Save the plot.
figure.savefig(path_results_root /
               'total_population_danes_vs_non_danes_pct.png')

Now, it is easy to see that proportions of Danes to non-Danes have changed between 2009 and 2020, and the the fraction of non-Danes is now bigger. It is, however, not easy to estimate the exact magnitude of the change. To make it possible, we once again turn to the interactive plotly plots.

#### Interactive graph with plotly

Let's create a copy of the graph above, and give the user the exact values for each bar when they hover the mouse pointer over it:

In [ ]:
# Get a dataframe only containing the representations in  %
df_danes_non_danes_pct = (
    df_danes_vs_non_danes[['% Danes', '% Non-danes']]
    .rename(columns={'% Danes': 'Danes',
                     '% Non-danes': 'Non-danes'})
    .reset_index()
    .melt(id_vars='Year',
          var_name='Origin',
          value_name='% of population')
)

# Create a figure containing the plotly bar graph.
fig = px.bar(
    df_danes_non_danes_pct,
    x='Year',
    y='% of population',
    color='Origin',
    barmode='group',
    color_discrete_sequence=px.colors.qualitative.D3
)

# Adjust the layout.
fig.update_layout(
    dict(plot_bgcolor='rgb(256, 256, 256)',
         paper_bgcolor='rgb(256, 256, 256)',
         title=dict(text="Population of Copenhagen",
                    x=0.5,
                    xanchor='center',
                    yanchor='top'),
         xaxis=dict(tickmode='linear',
#                     tick0=0.5,
                    dtick=1)
         )
)

# Show the figure.
fig.show()

# Save as HTML
fig.write_html(str(
    path_results_root / 'total_population_danes_vs_non_danes_pct.html'))

Now, we know exactly that the fraction of Danes has diminished from 78.4% in 2009 to 74.3% in 2020.

# Citizenship
<p style="border:2px solid black"> </p>

## Load data

Let's load the data containing information about the size of the population of Copenhagen by year and citizenship:

In [ ]:
# Load the data
cph_data = CphData()

# Get the citizenship database
df_citizenship = cph_data.citizenship
display(df_citizenship)

## Get percentage of district population by citizenship

To be able to better compare the districts, let's add a column where express the size of the population as a percentage of the total district population:

In [ ]:
# Get the district data by year.
df_citizenhip_by_district = (
    df_citizenship
    .loc[df_citizenship['District type'] == 'District']
    .groupby(get_df_columns(df_citizenship,
                            exclude=['Sex', 'Age', 'Number of people']),
             as_index=False)
    .sum()
    .drop(['Quarter', 'District type'], axis=1)
)

# Add the % of district population.
df_citizenhip_by_district['% of district population'] = (
    df_citizenhip_by_district
    .groupby(['Year', 'District'])['Number of people']
    .transform(np.sum)
    .div(df_citizenhip_by_district['Number of people'])
    .pow(-1)
    .mul(100)
)

# Show the dataframe.
display(df_citizenhip_by_district)

## Show data for 2020

To check if everything looks fine, let's view the data for 2020:

In [ ]:
# Define the data for 2020.
year = 2020

# Get the dataframe for the chosen year. 
# Let's sort by % of district popuplation.
df_citizenhip_by_district_2020 = (
    df_citizenhip_by_district
    .loc[df_citizenhip_by_district['Year'] == year]
    .sort_values(by=['Year', '% of district population'], ascending=False)
)

# Show the results
display(df_citizenhip_by_district_2020.head(15))

## Create a plotly barplot showing the percentages of population by citizenship for all districts in 2020

Now, let's visualize the data. We will do it using an interactive, grouped bar plot using plotly. We also sort the data so that the districts with the highest fraction of Danes will appear most to the left.

In [ ]:
# Create the plotly bar graph.
fig = px.bar(
    df_citizenhip_by_district_2020,
    x='District',
    y='% of district population',
    barmode='group',
    color='Citizenship',
    color_discrete_sequence=px.colors.qualitative.D3
)

# Update the layout of the figure.
fig.update_layout(
    dict(plot_bgcolor='rgb(256, 256, 256)',
         paper_bgcolor='rgb(256, 256, 256)',
         title=dict(text="Population of Copenhagen by district and citizenship",
                    x=0.5,
                    xanchor='center',
                    yanchor='top'),
         xaxis=dict(tickmode='linear',
                    #                         tick0=0.5
                    dtick=1),
         showlegend=True,
         xaxis_title=None
#          annotations=caption_attributes(caption, y),
#          margin=margin
         )
)

# Show the figure
fig.show()

## Create a geoplot of the percentage of non-Danes by district over time

Alright, let's do some animations, and try to visualize how the percentage of non-Danes have developed in different districts of Copenhagen over the years.

### All non-Danes

The non-Danes can be divided in Western non-Danes and non-Western non-Danes. Let's start by not differentiating between them.

In [ ]:
show_pct_non_danes_over_time(
    group='all',
    caption=None,
    caption_y=-0.05,
    margin=dict(r=0,
                t=30,
                l=0,
                b=0),
    range_color=(0, 25)
)

If we set the color range from 0 to 25% The visualization looks very dramatic. Especially Bispebjerg and Brjønshøj-Husum go into dark reds in 2017, but fade a little bit towards 2020. The drama is can be taken completely out of the animation by altering the color range to run from 0 to 100%, in which case one can barely trace the difference. I won't show it here, since I already present this effect in the [Explainer Notebook](https://social-data-analysis-and-visualization-final-project.s3.eu-central-1.amazonaws.com/explainer_notebook.html).

### Western non-Danes

Now, let's look only at the development of the population of Western non-Danes:

In [ ]:
show_pct_non_danes_over_time(
    group='western',
    caption=None,
    caption_y=-0.05,
    margin=dict(r=0,
                t=30,
                l=0,
                b=0),
    range_color=(0, 25)
)

Here, we can see that the Western non-Danes prefer central Copenhagen more than the other areas.

### Non-western non-Danes

And now, let's look at the non-western non-Danes:

In [ ]:
show_pct_non_danes_over_time(
    group='non-western',
    caption=None,
    caption_y=-0.05,
    margin=dict(r=0,
                t=30,
                l=0,
                b=0),
    range_color=(0, 25)
)

The non-Western non-Danes do not choose the same places to settle as their Western counterpart. From the animation we can deduct that their favorite location is Brønshøj-Husum.

# Marital status
<p style="border:2px solid black"> </p>

No analysis yet available.

# Family type and children
<p style="border:2px solid black"> </p>

No analysis yet available.

# Income
<p style="border:2px solid black"> </p>

Now, it's about money. Let's take a look at how much we earn!

## Load demographic data

Let's start by loading the dataframe containing the superset containing the combination of available data organized by year and district.

In [ ]:
# Load the dataframe containing all available data organized by year and district.
df_superset = pd.read_pickle(path_data_clean_root / 'cph_clean_superset.pkl')

# Show the dataframe.
display(df_superset)

## Load geodata

Load the geoshapes for the districts of Copenhagen from a geojson file.

In [ ]:
# Load the geo shapes from the json file.
district_geo_shapes = load_cph_district_geoshapes()

## Get data about the average income

Let's see the average income for men and women and add a column where we can see the difference.

In [ ]:
# Get the data about the average income.
df_income = (
    df_superset.loc[:, ['Year', 'District', 'Average income (kr.)']]
)

# Add information about the difference of income between sexes.
df_income['Difference between men and women'] =  (
    df_income[('Average income (kr.)', 'Men')]
    - df_income[('Average income (kr.)', 'Women')]
)

# Flatten the multiindex of the columns.
df_income = flatten_multiindex(df_income)
       
# Show the result.
display(flatten_multiindex(df_income))

## Show geoplot

Now let's show how the difference in the average income between the sexes has developed over time in different districts of Copenhagen:

In [ ]:
show_cph_geoplot(
    df_income,
    locations ='District',
    values = 'Difference between men and women',
    animation_frame='Year',
    label='',
    title='Difference in income between men and women [kr]',
    caption=''
)

Interestingly, we can see that the difference is biggest in the central Copenhagen and Østerbro, i.e. the districts with the highest income in Copenhagen. To me, this is not the result that I have expected and it would be interesting to investigate this observation a bit deeper.

# Life span
<p style="border:2px solid black"> </p>

No analysis yet available.

# Population movement data
<p style="border:2px solid black"> </p>

No analysis yet available.

# Dwellings
<p style="border:2px solid black"> </p>

No analysis yet available.